# Symmetric Eigenvalue Decomposition - Jacobi Method and High Relative Accuracy

---

The Jacobi method is the oldest method for EVD computations, dating back from 1864. 
The method does not require tridiagonalization.
Instead, the method computes a sequence of orthogonally similar 
matrices which converge to a diagonal matrix of eigenvalues. In each step a simple plane rotation
which sets one off-diagonal element to zero is performed. 

For positive definite matrices, the method computes eigenvalues with high relative accuracy.

For more details, see 
[I. Slapničar, Symmetric Matrix Eigenvalue Techniques][Hog14] and
[Z. Drmač, Computing Eigenvalues and Singular Values to High Relative Accuracy][Hog14a]
and the references therein.

[Hog14]: #1 "L. Hogben, ed., 'Handbook of Linear Algebra', pp. 55.1-55.25, CRC Press, Boca Raton, 2014."

[Hog14a]: #1 "L. Hogben, ed., 'Handbook of Linear Algebra', pp. 59.1-59.21, CRC Press, Boca Raton, 2014."


## Prerequisites

The reader should be familiar with concepts of eigenvalues and eigenvectors, related perturbation theory, and algorithms. 

 
## Competences 

The reader should be able to recognise matrices which warrant high relative accuracy and to apply 
Jacobi method to them.

---

## Jacobi method

$A$ is a real symmetric matrix of order $n$ and $A= U \Lambda  U^T$ is its EVD.

### Definitions

The __Jacobi method__ forms a sequence of matrices,
$$
A_0=A, \qquad A_{k+1}=G(i_k,j_k,c,s) A_k G(i_k,j_k,c,s)^T, \qquad
k=1,2,\ldots,
$$
where $G(i_k,j_k,c,s)$ is the orthogonal __plane rotation matrix__.
The parameters $c$ and $s$ are chosen such that 
$[A_{k+1}]_{i_k j_k}=[A_{k+1}]_{j_k i_k}=0$.

The plane rotation is also called the __Jacobi rotation__. 

The __off-norm__ of $A$ is 
$$
off(A)=\big(\sum_{i}\sum_{j\neq i} a_{ij}^2\big)^{1/2},
$$
that is, off-norm is the Frobenius norm of the
matrix consisting of all off-diagonal elements of $A$.

The choice of __pivot elements__ $[A_k]_{i_kj_k}$ is called the 
__pivoting strategy__.

The __optimal pivoting strategy__, originally used by Jacobi, chooses pivoting
elements such that $|[A_k]_{i_k j_k}|=\max_{i<j} |[A_k]_{ij}|$.

The __row-cyclic__ pivoting strategy chooses pivot elements
  in the systematic row-wise order,
$$
(1,2), (1,3), \ldots,(1,n),(2,3),
(2,4),\ldots,(2,n),(3,4),\ldots,(n-1,n).
$$
Similarly, the column-cyclic strategy chooses pivot elements column-wise.

One pass through all matrix elements is called __cycle__ or __sweep__.


### Facts

1. The Jacobi rotations parameters $c$ and $s$ are computed as follows:
 If $[A_k]_{i_kj_k}=0$, then $c=1$ and $s=0$, otherwise
$$
\tau=\frac{[A_k]_{i_ki_k}-[A_k]_{j_kj_k} }{2[A_k]_{i_kj_k} },\qquad
t=\frac{\mathop{\mathrm{sign}}(\tau)}{|\tau|+\sqrt{1+\tau^2}},\qquad
c=\frac{1}{\sqrt{1+t^2}},\qquad s=c\cdot t.
$$ 

2. After each rotation, the off-norm decreases,
$$
off^2(A_{k+1})=off^2(A_k)-2[A_k]_{i_kj_k}^2.
$$
With the appropriate pivoting strategy, the method converges in the sense that
$$
off(A_k)\to 0,\qquad A_k\to\Lambda, \qquad 
\prod_{k=1}^{\infty} G(i_k,j_k,c,s)^T \to U.
$$

3. For the optimal pivoting strategy
the square of the pivot element is greater than the average squared
element, $[A_k]_{i_kj_k}^2\geq
off^2(A) \frac{1}{n(n-1)}$. Thus,
$$
off^2(A_{k+1})\leq\left(1-\frac{2}{n(n-1)}\right)off^2(A_k)
$$
and the method converges.

4. For the row cyclic and the column cyclic pivoting strategies, the method
converges. The convergence is ultimately __quadratic__ in the sense that 
$off(A_{k+n(n-1)/2})\leq\ const\cdot  off^2(A_k)$, 
provided $off(A_k)$ is sufficiently small.

5. The EVD computed by the Jacobi method satisfies the standard error bounds.

6. The Jacobi method is suitable for parallel computation. There exist convergent parallel
strategies which enable simultaneous execution of several rotations.
  
7. The Jacobi method is simple, but it is slower than the methods based on tridiagonalization. It is
conjectured that standard implementations require $O(n^3\log n)$ operations. More precisely, each cycle clearly requires $O(n^3)$ operations and it is conjectured that $\log n$ cycles are needed until convergence.
 
8. If $A$ is positive definite, the method can be modified such that it reaches
the speed of the methods based on tridiagonalization and at the same time
computes the EVD with high relative accuracy.

### Examples

In [1]:
function myJacobi{T}(A::Array{T})
    n,m=size(A)
    U=eye(T,n)
    # Tolerance for rotation
    tol=sqrt(n)*eps(T)
    # Counters
    p=n*(n-1)/2
    sweep=0
    pcurrent=0
    # First criterion is for standard accuracy, second one is for relative accuracy
    # while sweep<30 && vecnorm(A-diagm(diag(A)))>tol
    while sweep<30 && pcurrent<p
        sweep+=1
        # Row-cyclic strategy
        for i = 1 : n-1 
            for j = i+1 : n
                # Check the tolerance - the first criterion is standard,
                # the second one is for relative accuracy for PD matrices               
                # if A[i,j]!=zero(T)
                if abs(A[i,j])>tol*sqrt(abs(A[i,i]*A[j,j]))
                    # Compute c and s
                    τ=(A[i,i]-A[j,j])/(2*A[i,j])
                    t=sign(τ)/(abs(τ)+sqrt(1+τ^2))
                    c=1/sqrt(1+t^2)
                    s=c*t
                    G=LinAlg.Givens(i,j,c,s)
                    A=G*A
                    # @show
                    A*=G'
                    A[i,j]=zero(T)
                    A[j,i]=zero(T)
                    U*=G'
                    pcurrent=0
                else
                    pcurrent+=1
                end
            end
        end
    end
    # λ, U
    # @show A
    diag(A), U
end

myJacobi (generic function with 1 method)

In [2]:
n=4
A=full(Symmetric(rand(n,n)))

4×4 Array{Float64,2}:
 0.978768  0.769325   0.585993   0.493777 
 0.769325  0.804012   0.188555   0.0583222
 0.585993  0.188555   0.974855   0.0510404
 0.493777  0.0583222  0.0510404  0.285318 

In [3]:
λ,U=myJacobi(A)

([2.07865,-0.18866,0.744397,0.40857],
[0.696533 -0.644716 -0.156179 0.273479; 0.499361 0.420936 -0.502536 -0.566488; 0.465318 0.230387 0.838969 -0.162885; 0.221268 0.595043 -0.13855 0.760111])

In [4]:
U'*U

4×4 Array{Float64,2}:
  1.0           9.38907e-17  -6.40948e-17  -7.387e-17  
  9.38907e-17   1.0          -1.93356e-16   2.48872e-17
 -6.40948e-17  -1.93356e-16   1.0          -1.04282e-17
 -7.387e-17     2.48872e-17  -1.04282e-17   1.0        

In [5]:
A*U-U*diagm(λ)

4×4 Array{Float64,2}:
  0.0          1.52656e-16   2.77556e-17  -8.32667e-17
 -2.22045e-16  1.52656e-16   2.77556e-16  -1.38778e-16
  0.0          1.59595e-16  -2.22045e-16  -8.32667e-17
  0.0          6.93889e-17   2.08167e-16  -1.11022e-16

In [6]:
# Positive definite matrix
n=100
A=rand(n,n)
A=full(Symmetric(A'*A));

In [7]:
λ,U=myJacobi(A)
norm(U'*U-I),norm(A*U-U*diagm(λ))

(2.8620697188713776e-14,3.9337990480549045e-11)

In [8]:
λ

100-element Array{Float64,1}:
    3.51855   
 2563.65      
    0.00100926
    0.056479  
    0.00366263
    0.0153288 
   30.0008    
    0.1662    
   22.9109    
   28.612     
    0.0982612 
   25.1767    
   26.6262    
    ⋮         
    7.33848   
    7.21591   
    4.93048   
    6.83255   
    6.35386   
    3.80823   
    5.17994   
    6.03175   
    5.5571    
    5.44693   
    6.91413   
    4.83968   

In [9]:
cond(A)

2.540133270029974e6

In [10]:
# Now the standard QR method
λ,U=eig(A);

In [11]:
norm(U'*U-I),norm(A*U-U*diagm(λ))

(4.238190776966289e-14,2.937623537812601e-12)

In [12]:
G=[1000 1;1 1]
H=[1001 1.001;1.001 1]
[eigvals(G) eigvals(H)]

2×2 Array{Float64,2}:
    0.998999     0.998998
 1000.0       1001.0     

## Relative perturbation theory

$A$  is a real symmetric PD matrix of order $n$  and $A=U\Lambda U^T$ is its EVD.

### Definition

The __scaled matrix__ of the matrix $A$ is the matrix
$$
A_S=D^{-1} A D^{-1}, \quad D=\mathop{\mathrm{diag}}(\sqrt{A_{11}},\sqrt{A_{22}},\ldots,\sqrt{A_{nn}}).
$$

### Facts

1. The above diagonal scaling is nearly optimal: 
$\kappa_2(A_S)\leq  n \min\limits_{D=\mathrm{diag}} \kappa(DHD) \leq n\kappa_2(H)$.

2. Let $A$ and $\tilde A=A+\Delta A$ both be positive definite, and let 
their eigenvalues have the same ordering. Then
$$
\frac{|\lambda_i-\tilde\lambda_i|}{\lambda_i}\leq 
\frac{\| D^{-1} (\Delta A) D^{-1}\|_2}{\lambda_{\min} (A_S)}\equiv
\|A_S^{-1}\|_2 \| \Delta A_S\|_2.
$$
If $\lambda_i$ and $\tilde\lambda_i$ are simple, 
$$
\|U_{:,i}-\tilde U_{:,i}\|_2 \leq \frac{\| A_S^{-1}\|_2 \|\Delta A_S\|_2}
{\displaystyle\min_{j\neq i}\frac{|\lambda_i-\lambda_j|}{\sqrt{\lambda_i\lambda_j}}}.
$$
These bounds are much sharper than the standard bounds for matrices for which $\kappa_2(A_S)\ll \kappa_2(A)$.

3. Jacobi method with the relative stopping criterion $|A_{ij}|\leq tol \sqrt{A_{ii}A_{jj}}$ for all $i\neq j$ and some user defined tolerance $tol$ (usually $tol=n\varepsilon$), computes the EVD with small scaled  backward error $\|\Delta A_S\|\leq \varepsilon\, O(\|A_S\|_2)\leq O(n)\varepsilon$, _provided_ that $\kappa_2([A_k]_S)$  does not grow much during the iterations. There is overwhelming numerical evidence that the scaled condition does not grow much, and the growth can be monitored, as well.

### Example - Scaled matrix


In [13]:
n=6
A=rand(n,n)
A=full(Symmetric(A'*A));
AS=map(Float64,[A[i,j]/sqrt(A[i,i]*A[j,j]) for i=1:n, j=1:n])

6×6 Array{Float64,2}:
 1.0       0.76255   0.841745  0.804963  0.921745  0.738822
 0.76255   1.0       0.66613   0.68334   0.779343  0.942937
 0.841745  0.66613   1.0       0.847525  0.723046  0.755997
 0.804963  0.68334   0.847525  1.0       0.693296  0.643683
 0.921745  0.779343  0.723046  0.693296  1.0       0.684932
 0.738822  0.942937  0.755997  0.643683  0.684932  1.0     

In [14]:
cond(AS), cond(A)

(693.8621331087436,615.8538660024076)

In [15]:
# Strong scaling
D=exp(50*(rand(n)-0.5))

6-element Array{Float64,1}:
     2.50597e7 
 11856.8       
     0.00148747
 18871.8       
     2.84056e8 
     7.56253   

In [16]:
H=diagm(D)*AS*diagm(D)

6×6 Array{Float64,2}:
     6.27988e14      2.26574e11  …  6.56131e15      1.40018e8 
     2.26574e11      1.40583e8      2.62482e12  84550.6       
 31376.5            11.7483         3.05505e5       0.00850425
     3.80685e11      1.52904e8      3.71652e12  91865.7       
     6.56131e15      2.62482e12     8.06877e16      1.47136e9 
     1.40018e8   84550.6         …  1.47136e9      57.1918    

In [17]:
cond(H)

6.849577395214291e23

In [18]:
λ,U=myJacobi(H)

([9.38192e13,4.72363e7,1.18772e-7,1.25899e8,8.12219e16,4.14681],
[0.996702 9.851e-5 … 0.0811435 -1.70751e-7; 0.000136862 0.960218 … 3.24364e-5 -0.00065635; … ; -0.0811435 -2.63846e-5 … 0.996702 1.44339e-8; 2.14932e-7 0.000645804 … 1.81954e-8 1.0])

In [19]:
λ1,U1=eig(H)

([8.12219e16,9.38192e13,1.25899e8,4.72363e7,4.14681,1.1831e-7],
[-0.0811435 0.996702 … 1.70751e-7 8.7976e-12; -3.24364e-5 0.000136862 … 0.00065635 1.82471e-7; … ; -0.996702 -0.0811435 … -1.44339e-8 -2.93476e-12; -1.81954e-8 2.14932e-7 … -1.0 -0.000282484])

In [20]:
[sort(λ) sort(λ1)]

6×2 Array{Float64,2}:
 1.18772e-7  1.1831e-7 
 4.14681     4.14681   
 4.72363e7   4.72363e7 
 1.25899e8   1.25899e8 
 9.38192e13  9.38192e13
 8.12219e16  8.12219e16

In [21]:
# Check with BigFloat
λ2,U2=myJacobi(map(BigFloat,H))

(BigFloat[9.381916422813170705987907680214493045648268495030343585313210604534376900250206e+13,4.723634347112508327023998020098545803459134622176081005023378462751769391746603e+07,1.187715611345956048981186615200826960571930414382767648227924480318136806521461e-07,1.258988887717428806933572805519883746768385366053440914979266722981097006617094e+08,8.12218506114917101382627083554788551702538944570186805595405624092410830982113e+16,4.146807834211247393972014542409443027041209981514681772539653306319349059044546],
BigFloat[9.967020726258624951144094946678498221524312393548271584106230425835625421773045e-01 9.850996728779473488017908216772417199572351574855738981062800817081878965018677e-05 … 8.114350707366643855754987977745495213862581622017533686801557059041317485228134e-02 -1.707513322805917910138510943687973701696566758284260344948207314957232961664452e-07; 1.368622540672325436809430317024330728731755411813688309743626036720275139533417e-04 9.6021810012748546935665898755964776644322540

In [22]:
# Relative error is eps()*cond(AS)
(sort(λ2)-sort(λ))./sort(λ2)

6-element Array{BigFloat,1}:
 -3.262964344527058952990233138476527985630273920873541441321900853538081126901286e-15
 -4.320189028087171308040348269864681491603421024969806404373633787182026076405611e-15
 -4.289711375953355924871947750027288352821241973432067255525998334049043822109968e-16
 -4.706608528643247365792207076222769438961993158234909248912272217514620473419375e-16
 -7.907778920604786528323117958632241354754959653484275403373680682263266452758186e-16
 -8.108869324669886746216799969164226590235751972668548791971044486316299754436395e-16

## Indefinite matrices

### Definition

__Spectral absolute value__ of the matrix $A$ is the matrix is $|A|_{spr}=(A^2)^{1/2}$ (positive definite part of the polar decomposition of $A$).

### Facts

1. The above perturbation bounds for positive definite matrices essentially hold with $A_S$ replaced by $[|A|_{spr}]_S$.

2. Jacobi method can be modified to compute the EVD with small backward error 
$\| \Delta [|A|_{spr}]_S\|_2$.

The details of the indefinite case are beyond the scope of this course, and the reader should consider references.